In [1]:
import pandas as pd
import numpy as np

In [2]:
num = pd.read_csv("numerical(1).csv")
num.shape

(95412, 315)

In [3]:
cat = pd.read_csv("categorical.csv")
cat.shape

(95412, 22)

In [4]:
#Converting DATASRCE into object
cat["DATASRCE"] = cat["DATASRCE"].astype('object')

In [5]:
# Dropping RFA_2R because it has only one unique value
cat.drop("RFA_2R",axis =1,inplace=True)

In [6]:
target = pd.read_csv("target.csv")

In [7]:
X = pd.concat([num,cat],axis =1)

In [8]:
X_concat = pd.concat([X,target],axis=1)

In [9]:
X_concat.shape

(95412, 338)

# Only look at people who have donated (Target B = 1)

In [10]:
new_X = X_concat[X_concat["TARGET_B"] == 1]
new_X.shape

(4843, 338)

In [11]:
#Dropping TARGET_B
new_X.drop(["TARGET_B"],axis=1,inplace = True)

C:\Users\Denish\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
new_X

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_D
20,2,62.000000,3,8,10,2,25,40,27,11,...,1,88,1,94,4,96,3,87,1,4.0
30,0,61.611649,5,9,0,1,37,58,16,8,...,2,90,4,93,1,95,12,90,4,7.0
45,0,66.000000,5,9,5,0,33,24,39,6,...,10,93,12,94,4,96,2,87,4,5.0
78,0,69.000000,6,9,0,0,34,20,54,2,...,7,90,1,95,3,95,11,90,1,13.0
93,1,73.000000,1,7,10,0,21,53,8,5,...,10,92,9,95,9,95,9,92,9,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95298,2,45.000000,5,9,0,0,45,28,37,9,...,4,89,6,96,1,96,1,86,8,20.0
95309,0,51.000000,5,6,1,1,32,43,24,7,...,1,93,10,94,2,95,12,93,10,15.0
95398,0,86.000000,5,9,0,1,32,21,26,9,...,10,89,6,95,11,96,2,87,11,3.0
95403,0,58.000000,4,9,0,0,24,46,20,6,...,1,90,3,93,12,96,1,90,3,10.0


In [13]:
y = new_X["TARGET_D"]
new_X = new_X.drop(["TARGET_D"],axis=1)

In [14]:
y = y.reset_index(drop=True)
# new_y = new_y.reset_index(drop=True)
# new_y = pd.DataFrame({'TARGET_D': y})
y.shape

(4843,)

In [15]:
new_X = new_X.reset_index(drop = True)
new_X.shape

(4843, 336)

In [16]:
X_num = new_X.select_dtypes(np.number)
X_cat = new_X.select_dtypes(include = "object")

# Feature Selection on Numerical Columns

In [17]:
print("Total no of columns before FS: " ,X_num.shape)

Total no of columns before FS:  (4843, 329)


In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_num)
numerical_scaled = pd.DataFrame(scaler.fit_transform(X_num),columns = X_num.columns)

In [311]:
numerical_scaled.shape

(4843, 329)

In [312]:
from sklearn.feature_selection import SelectKBest, f_regression
import pandas as pd
from sklearn.feature_selection import SelectKBest, mutual_info_regression

# Assuming numerical_scaled is your feature matrix and y is your target

# Initialize SelectKBest
selector = SelectKBest(score_func= f_regression, k=30)

# Fit and transform on training data
X_FS_new = selector.fit_transform(numerical_scaled, y)

# Get the selected feature indices
selected_indices = selector.get_support(indices=True)

# Get the names of selected features
selected_features = numerical_scaled.columns[selected_indices]

print("Selected features:")
print(selected_features)

# Assuming your target is numerical for a regression task
K_Best_Num = pd.DataFrame(X_FS_new, columns=selected_features)


Selected features:
Index(['HV1', 'HV2', 'HVP1', 'HVP2', 'HVP3', 'HVP4', 'HVP6', 'IC1', 'IC2',
       'IC3', 'IC4', 'IC5', 'IC12', 'IC20', 'IC21', 'EC4', 'EC8', 'RAMNTALL',
       'NGIFTALL', 'CARDGIFT', 'MINRAMNT', 'MAXRAMNT', 'LASTGIFT', 'TIMELAG',
       'AVGGIFT', 'RFA_2F', 'CLUSTER2', 'ODATEW_YR', 'MAXRDATE_YR',
       'FIRSTDATE_YR'],
      dtype='object')


In [313]:
X_cat = X_cat.reset_index(drop = True)


In [314]:
X_NEW_NUM_CAT = pd.concat([K_Best_Num,X_cat],axis=1)


# Train Test Split

In [315]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_NEW_NUM_CAT, y, test_size=0.30, random_state = 1)

In [316]:
# X_train and X_test already sclaled before feature selection
#Xonly categorical data needs to be encoded

In [317]:
train_cat = X_train.select_dtypes(include = "object")
train_num = X_train.select_dtypes(exclude = "object")
test_cat = X_test.select_dtypes(include = "object")
test_num = X_test.select_dtypes(exclude = "object")

# train_cat = X_train.select_dtypes(include = "object")
# test_cat = X_test.select_dtypes(include = "object")

In [318]:
# Encode the categorical features using One-Hot Encoding

# One hot encoding
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(train_cat)

cols = encoder.get_feature_names_out(input_features=train_cat.columns)

#X_train_one_hot = encoder.transform(train_cat).toarray()
X_train_one_hot = pd.DataFrame(encoder.transform(train_cat).toarray(),columns=cols)
X_test_one_hot= pd.DataFrame(encoder.transform(test_cat).toarray(),columns=cols)

# Concating training and testing data

In [319]:
train_num = train_num.reset_index(drop = True)
test_num = test_num.reset_index(drop = True)

In [320]:
X_train_one_hot

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3385,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3387,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [321]:
X_train_r = pd.concat((train_num,X_train_one_hot),axis=1)
X_test_r = pd.concat((test_num,X_test_one_hot),axis=1)

# Linear Regression


In [322]:
y_train = y_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)
X_train_r =X_train_r.reset_index(drop = True)

In [323]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

linear_model = LinearRegression()
linear_model.fit(X_train_r, y_train)

# Make predictions on the entire dataset
y_pred = linear_model.predict(X_test_r)

# Evaluate the model
print("y_pred:", y_pred)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)


r2_linear_reg = r2_score(y_test, y_pred)
print("R2 Score: ",r2_linear_reg)


y_pred: [ 7.51446605  6.76891022  2.85950528 ... 22.46264836  4.59651104
 30.13113217]
Mean Squared Error: 143.50524353653384
Mean Absolute Error: 4.508662887740496
R2 Score:  0.14064884873916073


In [324]:
r2_linear_reg

0.14064884873916073

In [325]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [326]:

from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor()
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
from sklearn.neighbors import KNeighborsRegressor
model3 = KNeighborsRegressor(n_neighbors=10)
from sklearn.ensemble import RandomForestRegressor
model4 =  RandomForestRegressor(max_depth=20, random_state=0)

In [327]:
from sklearn.model_selection import cross_validate
model_pipeline = [model1, model2, model3,model4]
model_names = ['Decision Tree Regressor', 'Linear Regression', 'KNN', "Random Forest"]
scores = {}
for model, model_name in zip(model_pipeline, model_names):
    print(model_name)
    #model.fit( X_train_r,y_train)
    scores_res = cross_validate(model, X_train_r, y_train, cv=7, scoring = ['r2', 'neg_mean_squared_error'])
    scores[model_name] = scores_res
    print(f"r2: {scores[model_name]['test_r2'].mean()}")
    print(f"mse: {scores[model_name]['test_neg_mean_squared_error'].mean()}")
#print(scores)


Decision Tree Regressor
r2: 0.03965380683524294
mse: -141.06756091146437
Linear Regression
r2: 0.5945110409805894
mse: -60.58198882742698
KNN
r2: 0.3701072767004318
mse: -94.63394369523485
Random Forest
r2: 0.5349504551826352
mse: -69.0114196017187


# Random Forest

In [328]:
from sklearn.ensemble import RandomForestRegressor


In [329]:
regr = RandomForestRegressor(max_depth=12, random_state=0)
regr.fit(X_train_r, y_train)
mean_score_RF = np.mean(cross_val_score(regr, X_train_r, y_train, cv=7))


In [330]:
mean_score_RF

0.5355171165316801

In [331]:
y_pred_RF = regr.predict(X_test_r)
print(regr.predict(X_test_r))

[ 8.891416    4.03001524  3.79196949 ... 20.70798701  8.32501436
 31.75122516]


In [332]:
from sklearn.metrics import r2_score
mse = mean_squared_error(y_test, y_pred_RF)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred_RF)
print("Mean Absolute Error:", mae)

r2 = r2_score(y_test, y_pred_RF)
print("R2 Score: ",r2)

Mean Squared Error: 111.94270851307641
Mean Absolute Error: 4.49881927806016
R2 Score:  0.32965449160414495


# Next Part of the PRoject

In [333]:
#Using the regression model,make predictions on all of the people our classification model predicted will donat

In [334]:
testing_data = pd.read_csv("testing_data_previous_lab_Target_B.csv")

In [335]:
testing_data.drop("Unnamed: 0",axis=1,inplace = True)

In [336]:
testing_Data_FS_num = []
for i in X_num.columns:
    if i in testing_data.columns:
        testing_Data_FS_num.append(i)
    

In [337]:
testing_Data_FS_cat = []
for i in testing_data.columns:
    if i not in testing_Data_FS_num:
        testing_Data_FS_cat.append(i)

In [338]:
# Note: Here i have converted main testing dataframe into numerical and categorical required by this lab 
#testing_Data_FS_num --- all the numerical columns
#testing_Data_FS_cat ---- all the categorical columns

In [339]:
testing_Data_num = testing_data[testing_Data_FS_num]

# applying FS on test data

In [340]:
from sklearn.preprocessing import MinMaxScaler

testing_Data_scaled = pd.DataFrame(scaler.transform(testing_Data_num),columns = testing_Data_num.columns)

In [341]:
X_test_new = selector.transform(testing_Data_scaled)

# Get the names of selected features
selected_features_test = testing_Data_scaled.columns[selected_indices]

print("Selected features:")
print(selected_features)


test_k_Best_Num = pd.DataFrame(X_test_new,columns = selected_features_test)


Selected features:
Index(['HV1', 'HV2', 'HVP1', 'HVP2', 'HVP3', 'HVP4', 'HVP6', 'IC1', 'IC2',
       'IC3', 'IC4', 'IC5', 'IC12', 'IC20', 'IC21', 'EC4', 'EC8', 'RAMNTALL',
       'NGIFTALL', 'CARDGIFT', 'MINRAMNT', 'MAXRAMNT', 'LASTGIFT', 'TIMELAG',
       'AVGGIFT', 'RFA_2F', 'CLUSTER2', 'ODATEW_YR', 'MAXRDATE_YR',
       'FIRSTDATE_YR'],
      dtype='object')


In [342]:
testing_Data_cat = testing_data[testing_Data_FS_cat]

In [343]:
test_data_ready = pd.concat((test_k_Best_Num,testing_Data_cat),axis=1)

In [344]:
y_pred_test = linear_model.predict(test_data_ready)

In [345]:
y_pred_RF_test = regr.predict(test_data_ready)

In [346]:
kn = KNeighborsRegressor(n_neighbors=10)
kn.fit(X_train_r, y_train)
y_pre_kn_test = kn.predict(test_data_ready)


In [365]:
#Following is the result of three algorithm predicting total donor amount
print("Random Forest: ",y_pred_RF_test)
print("linear regression: ",y_pred_test)
print("KNeighborsRegressor: ",y_pre_kn_test)

Random Forest:  [ 7.59255375  7.55531566  7.55880375 ... 10.04545455  8.11323232
  7.52856566]
linear regression:  [-5.99394984 -7.60714949 -6.93155527 ... -4.74324097 -7.5888851
 -7.25117019]
KNeighborsRegressor:  [8.8 5.3 6.5 ... 9.8 9.8 7.1]


# Conclusion

In [348]:
total_num_donors = 72464 

In [363]:
# Analysis from the given data
print("Total donation given : ",target[target["TARGET_B"] == 1]["TARGET_D"].sum()) 
print("Total no of donors: ",len(new_X))
print("Cost of total git: ",len(new_X)*0.68)

Total donation given :  75668.7
Total no of donors:  4843
Cost of total git:  3293.2400000000002


In [366]:
# Analysis of prediction
print("Total donation predicted RF: ",y_pred_RF_test.sum()) 
print("Total donation predicted LR: ",y_pred_test.sum()) 
print("Total donation predicted KN: ",y_pre_kn_test.sum()) 
print("Total no of donors: ",len(test_data_ready))
print("Cost of total git: ",len(test_data_ready)*0.68)

Total donation predicted RF:  652268.8607569941
Total donation predicted LR:  -415488.3841781858
Total donation predicted KN:  746423.7
Total no of donors:  76204
Cost of total git:  51818.72


In [ ]:
# There is so much prediction error we can further improve the performance by changing the hyperparameters of model 
#and changing the value of k in feature selection